# Especialização em Inteligência Artificial – IFMG
# Trabalho da disciplina de Redes Neurais e Aprendizado Profundo
Autor: Alexandre Fortes Santana

Professor: Agnaldo José da Rocha Reis - UFOP

### 1. O que é inteligência para você?

Para mim, inteligência poderia ser definida pelos tópicos a seguir:
- Inteligência se manifesta em diferentes graus e tipos;
- Inteligência se desenvolve em um indivíduo, a princípio, biológico;
- O indivíduo detentor de inteligência em questão precisa ter a capacidade de memorizar informações em alguma escala;
- O indivíduo detentor de inteligência utiliza as informações a que tem acesso para interpretar o mundo a sua volta e a si mesmo;
- O indivíduo detentor de inteligência é capaz de decisões, agir e criar novos artefatos (imaginários ou físicos).

### 2. Em sua opinião, o que aconteceria se alguém descobrisse como implementar uma IA mais abrangente (e.g., AGI) em um robô?
A descoberta de uma Inteligência Artificial Geral seria um marco histórico para a humanidade. Experimentaríamos um período de frenesi nas redes sociais, nos noticiários e nas rodas de conversa. Muitos dilemas seriam discutidos, abordando temas como mercado de trabalho, impacto social, segurança, regulação, ética e questões militares. Os primeiros robôs focariam em demonstrar o potencial de suas aplicações e em realizar apresentações que alimentassem o frenesi público. Após um período marcado por medo, especulações e empolgação, veríamos as primeiras aplicações práticas direcionadas a problemas reais. Os primeiros robôs comerciais seriam mais simples, devido ao elevado custo de produção, e não necessariamente seriam humanoides. Não, não acredito que as máquinas se revoltariam, levando a um apocalipse.

### 3. Análise de um processo de destilação fracionada de petróleo
A partir da análise de um processo de destilação fracionada de petróleo observou-se que determinado óleo poderia ser classificado em duas classes de pureza {C1 e C2}, mediante a medição de três grandezas {x1, x2 e x3} que representam algumas das propriedades físico-químicas do óleo. Para tanto, pretende-se utilizar um perceptron para executar a classificação automática dessas duas classes. Assim, baseadas nas informações coletadas do processo, formou-se o conjunto de treinamento em anexo (vou te passar a estrutura de dados nas próximas mensagens), tomando por convenção o valor –1 para óleo pertencente à classe C1 e o valor +1 para óleo pertencente à classe C2.

**a. Execute dois treinamentos para a rede perceptron, inicializando-se o vetor de pesos em cada treinamento com valores aleatórios entre zero e um de tal forma que os elementos do vetor de pesos iniciais não sejam os mesmos.**

In [246]:
import numpy as np
import pandas as pd

data = np.loadtxt('tab_treinamento1.dat')
training_data = data[:, :3]
labels = data[:, 3]
#print(data)
#print(training_data)
#print(labels)

# Bias: Adicionando uma coluna de uns ao conjunto de dados de treinamento
training_data = np.c_[np.ones(training_data.shape[0]), training_data]

epochs = 1000
learning_rate = 0.1

# Inicialização de pesos
def initialize_weights(dim):
    return np.random.rand(dim)

# Treinamento do Perceptron
def train_perceptron(training_data, labels, learning_rate, epochs):
    # Inicialização de pesos
    weights = initialize_weights(training_data.shape[1])
    initial_weights = np.copy(weights)
    no_errors = 0
    final_epoch = 0
    
    for epoch in range(epochs):
        for i in range(len(training_data)):
            x = training_data[i]
            y = labels[i]
            
            # Cálculo do output e função de ativação
            output = np.dot(weights, x)
            prediction = 1 if output > 0 else -1

            #print("y:", y, "prediction:", prediction)

            # Atualização de pesos
            if prediction != y:
                weights += learning_rate * (y - prediction) * x
                no_errors += 1

        if no_errors == 0:
            final_epoch = epoch+1
            print(f"Convergiu na época {final_epoch}")
            break
        no_errors = 0 # reseta contador de erros
                
    return initial_weights, weights, final_epoch

# DataFrame para armazenar os resultados
results_df = pd.DataFrame(columns=["Treinamento", "Vetor de Pesos Inicial", "Vetor de Pesos Final", "Número de Épocas"])

# Executando Dois Treinamentos
for i in range(2):
    initial_weights, final_weights, final_epoch = train_perceptron(training_data, labels, learning_rate, epochs)  
    
    print(initial_weights)
    print(final_weights)
    print(final_epoch)

    new_row_df = pd.DataFrame({
        "Treinamento": f"T{i+1}",
        "Vetor de Pesos Inicial": initial_weights,
        "Vetor de Pesos Final": final_weights,
        "Número de Épocas": final_epoch
    })
    results_df = pd.concat([results_df, new_row_df], ignore_index=True)

results_df

Convergiu na época 433
[0.87132654 0.55759443 0.15687519 0.4031135 ]
[31.67132654 16.02395443 25.45669519 -7.5308065 ]
433
Convergiu na época 413
[0.46835909 0.13546439 0.47693809 0.48523506]
[31.06835909 15.60934439 25.07245809 -7.38660494]
413


,Treinamento,Vetor de Pesos Inicial,Vetor de Pesos Final,Número de Épocas
0,T1,0.871327,31.671327,433
1,T1,0.557594,16.023954,433
2,T1,0.156875,25.456695,433
3,T1,0.403114,-7.530806,433
4,T2,0.468359,31.068359,413
5,T2,0.135464,15.609344,413
6,T2,0.476938,25.072458,413
7,T2,0.485235,-7.386605,413


In [248]:
grouped = results_df.groupby("Treinamento")["Vetor de Pesos Final"].apply(list).reset_index()

weights_T1 = np.array(grouped[grouped["Treinamento"] == "T1"]["Vetor de Pesos Final"].iloc[0])
weights_T2 = np.array(grouped[grouped["Treinamento"] == "T2"]["Vetor de Pesos Final"].iloc[0])
#print(weights_T1)
#print(weights_T2)

[31.67132654 16.02395443 25.45669519 -7.5308065 ]
[31.06835909 15.60934439 25.07245809 -7.38660494]


**c. Após o treinamento do perceptron, aplique-o na classificação automática de novas amostras de óleo (ver arquivo tab_teste1.dat), indicando-se na tabela seguinte os resultados das saídas (Classes) referentes aos dois processos de treinamento realizados no item a.**

Para classificar novas amostras usando os pesos finais obtidos após o treinamento vamos criar uma função chamada classify_samples que receberá as novas amostras e os pesos finais.

In [256]:
new_samples = np.loadtxt('tab_teste1.dat')
new_samples
# Função para classificar novas amostras
def classify_samples(samples, weights):
    predictions = np.sign(np.dot(samples, weights[1:]) + weights[0])  # Operação vetorizada
    return predictions

# Assumindo que trained_weights é uma lista dos vetores de pesos finais de cada treinamento
trained_weights = [weights_T1, weights_T2]  # Exemplo, substituir pelos seus pesos reais
print(trained_weights)

# Classificar novas amostras para cada treinamento
for i, weights in enumerate(trained_weights):
    predictions = classify_samples(new_samples, weights)
    print(f"Classificações usando o treinamento T{i+1}: {predictions}")

[array([31.67132654, 16.02395443, 25.45669519, -7.5308065 ]), array([31.06835909, 15.60934439, 25.07245809, -7.38660494])]
Classificações usando o treinamento T1: [-1.  1.  1.  1.  1.  1. -1.  1. -1. -1.]
Classificações usando o treinamento T2: [-1.  1.  1.  1.  1.  1. -1.  1. -1. -1.]
